In [1]:
from datetime import datetime, timedelta
import pandas as pd
from dataclasses import dataclass
from typing import Callable

data_folder = '../data/lorentzian'

@dataclass
class YahooDataSettings:
    ticker:str = 'BTC-GBP'
    interval:str = '1d'
    window_in_days:int = 365
    offset_in_days:int = 0

@dataclass
class KrakenCSVSettings:
   pair:str = 'XBTGBP'
   interval_in_min:int = 1440
   window_in_intervals: int = 365
   windows_to_skip: int = 0

# symbol, interval, window, offset, buffer
GetDataFunction = Callable[[str, str, int, int, int], None]

@dataclass
class DataSettings:
   symbol:str
   interval:str
   window: int
   skip: int
   buffer: int
   get_data: GetDataFunction

@dataclass
class LorentzianSettings:
    neighborsCount:int = 8
    maxBarsBack:int = 2000
    useDynamicExits:bool = False
    useEmaFilter:bool = False
    emaPeriod:int = 200
    useSmaFilter:bool = False
    smaPeriod:int = 200
    useKernelSmoothing:bool = False
    lookbackWindow:int = 8
    relativeWeight:float = 8.0
    regressionLevel:int = 25
    crossoverLag:int = 2
    useVolatilityFilter:bool = False
    useRegimeFilter:bool = False
    useAdxFilter:bool = False
    regimeThreshold:float = 0.0
    adxThreshold:int = 0
    use_RSI:bool = True
    RSI_param1:int = 14
    RSI_param2:int = 2
    use_WT:bool = True 
    WT_param1:int = 10
    WT_param2:int = 11
    use_CCI:bool = True
    CCI_param1:int = 20
    CCI_param2:int = 2
    use_ADX:bool = True
    ADX_param1:int = 20
    ADX_param2:int = 2
    use_MFI:bool = True
    MFI_param1:int = 14

@dataclass
class StrategySettings:
  transcaction_costs:float = .004
  pot_percentage_to_bet:float = 1.0
  initial_cash_percentage:float = 1.0
  initial_value: float = 1000.0
  stop_loss_percentage: float = .1

@dataclass
class StrategyReturns:
  initial_value: float
  final_value: float
  portfolio_growth: float
  asset_growth: float
  trade_count: int
  win_percentage: float
  activity_log: any

@dataclass
class LorentzianPredictionResult:
    data_settings: YahooDataSettings
    settings: LorentzianSettings
    returns: StrategyReturns

In [3]:
import yfinance

def get_data_from_yfinance(symbol: str, interval: int, window: int, skip: int, buffer: int):
    start_time = datetime.now() - timedelta(days = window + window * skip + buffer)
    end_time = datetime.now() - timedelta(days = window * skip)

    ticker_data = yfinance.Ticker(symbol)

    df = ticker_data.history(interval=interval, start=start_time, end=end_time)

    new_names = {'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 'Volume': 'volume'}
    df = df.rename(columns=new_names)

    df.index = df.index.tz_convert('UTC')
    
    return df

yfinance_df = get_data_from_yfinance('BTC-GBP', '1h', 24, 0, 0)
yfinance_df

,open,high,low,close,volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-05-16 22:00:00+00:00,51675.683594,51675.683594,51434.488281,51611.875000,0,0.0,0.0
2024-05-16 23:00:00+00:00,51629.589844,51632.367188,51463.335938,51463.335938,0,0.0,0.0
2024-05-17 00:00:00+00:00,51486.957031,51707.835938,51486.957031,51707.835938,0,0.0,0.0
2024-05-17 01:00:00+00:00,51671.062500,51706.554688,51452.917969,51621.304688,0,0.0,0.0
2024-05-17 02:00:00+00:00,51625.476562,51722.406250,51510.429688,51666.710938,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-09 18:00:00+00:00,54843.171875,54948.468750,54822.496094,54898.925781,213783552,0.0,0.0
2024-06-09 19:00:00+00:00,54895.214844,54961.210938,54740.460938,54792.890625,248645632,0.0,0.0
2024-06-09 20:00:00+00:00,54793.656250,54823.464844,54666.269531,54740.136719,144299008,0.0,0.0


In [497]:
base_kraken_path = '../data/kraken'

def get_data_from_kraken_csv(symbol: str, interval: str, window: int, skip: int, buffer: int):
    column_names = ['date_unix', 'open', 'high', 'low', 'close', 'volume', 'trades']
    df = pd.read_csv(f'{base_kraken_path}/{symbol}_{interval}.csv', names=column_names)
    
    df['date'] = pd.to_datetime(df['date_unix'], unit='s')
    del df['date_unix']
    df = df.set_index('date')

    start_index = max(len(df) - window - buffer - window * skip, 0)
    end_index = max(len(df) - window * skip, 0)
    return df.iloc[start_index:end_index]

kraken_df = get_data_from_kraken_csv('XBTGBP', '1440', 365, 3, 2000)
kraken_df

,open,high,low,close,volume,trades
date,,,,,,
2014-11-06,213.00000,213.00,213.00000,213.0,0.060000,2
2014-11-07,215.00000,215.00,215.00000,215.0,1.000000,1
2014-11-08,215.00000,215.00,215.00000,215.0,0.405000,1
2014-11-10,223.00000,224.00,223.00000,224.0,0.200000,2
2014-11-12,269.99999,274.99,269.99999,271.0,0.942017,6
...,...,...,...,...,...,...
2021-03-28,40602.10000,41053.70,39800.10000,40556.4,85.230638,3405
2021-03-29,40553.80000,42337.90,40002.40000,41885.3,125.533278,2786
2021-03-30,41874.20000,43200.00,41452.60000,42773.1,89.446488,2133


In [498]:
from advanced_ta import LorentzianClassification
from ta.volume import money_flow_index as MFI

def calculate_predictions(df, settings: LorentzianSettings):
    features = []
    if settings.use_RSI:
        features.append(LorentzianClassification.Feature("RSI", settings.RSI_param1, settings.RSI_param2))
    if settings.use_WT:
        features.append(LorentzianClassification.Feature("WT", settings.WT_param1, settings.WT_param2))
    if settings.use_CCI:
        features.append(LorentzianClassification.Feature("CCI", settings.CCI_param1, settings.CCI_param2))
    if settings.use_ADX:
        features.append(LorentzianClassification.Feature("ADX", settings.ADX_param1, settings.ADX_param2))
    if settings.use_MFI:
        features.append(MFI(df['high'], df['low'], df['close'], df['volume'], settings.MFI_param1))

    # df here is the dataframe containing stock data as [['open', 'high', 'low', 'close', 'volume']]
    lc = LorentzianClassification(
        df,    
        features,
        settings=LorentzianClassification.Settings(
            source='close',
            neighborsCount=settings.neighborsCount,
            maxBarsBack=settings.maxBarsBack,
            useDynamicExits=settings.useDynamicExits
        ),
        filterSettings=LorentzianClassification.FilterSettings(
            useVolatilityFilter=settings.useVolatilityFilter,
            useRegimeFilter=settings.useRegimeFilter,
            useAdxFilter=settings.useAdxFilter,
            regimeThreshold=settings.regimeThreshold,
            adxThreshold=settings.adxThreshold,
            kernelFilter = LorentzianClassification.KernelFilter(
                useKernelSmoothing = settings.useKernelSmoothing,
                lookbackWindow = settings.lookbackWindow,
                relativeWeight = settings.relativeWeight,
                regressionLevel = settings.regressionLevel,
                crossoverLag = settings.crossoverLag
            )
        )
    )

    return lc

In [675]:
def calculate_performance(df, settings: StrategySettings) -> StrategyReturns: 
  first_price = df.iloc[0]['open']
  last_price = df.tail(1).iloc[0]['open']
  max_price = df['high'].max()
  min_price = df['low'].min()
  price_range = max_price - min_price

  transcaction_costs = settings.transcaction_costs
  pot_percentage_to_bet = settings.pot_percentage_to_bet
  initial_cash_percentage = settings.initial_cash_percentage

  initial_value = settings.initial_value * 100
  initial_cash = (initial_value * initial_cash_percentage)
  initial_shares = ((initial_value * (1- initial_cash_percentage)) * (1 - transcaction_costs)) / first_price
  
  activity_log = []
  activity_log.append(f'Initial cash is £{initial_cash / 100} and initial shares are {initial_shares}')
  activity_log.append('---------------------')

  cash = initial_cash
  shares = initial_shares

  is_long = True
  trade_size = 0
  trade_value = 0
  trade_count = 0
  trade_price = 0
  win_count = 0

  for index, row in df.iterrows():
    price = row['close']

    if trade_size > 0 and is_long and (not pd.isna(row['endLongTrade']) or trade_price > price * (1 - settings.stop_loss_percentage)):
      shares -= trade_size
      cash_value = trade_size * price * (1 - transcaction_costs)
      cash += cash_value
      trade_count += 1
      if (cash_value > trade_value):
        win_count += 1

      activity_log.append(f'{index} - Closed long trade for {trade_size} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')

      trade_size = 0
      trade_value = 0
    elif trade_size > 0 and not is_long and (not pd.isna(row['endShortTrade']) or trade_price < price * (1 + settings.stop_loss_percentage)):
      shares += trade_size
      cash_value = trade_size * price * (1 + transcaction_costs)
      cash -= cash_value
      trade_count += 1
      if (cash_value < trade_value):
        win_count += 1

      activity_log.append(f'{index} - Closed short trade for {shortTradeSize} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')

      trade_size = 0
      trade_value = 0
    elif not pd.isna(row['startLongTrade']) and trade_size == 0:
      cash_value = cash * pot_percentage_to_bet
      cash -= cash_value
      trade_size = (cash_value * (1 - transcaction_costs)) / price
      shares += trade_size
      is_long = True
      trade_value = cash_value
      trade_price = price

      activity_log.append(f'{index} - Opened long trade for {trade_size} shares at {price}. Paid £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')
    elif not pd.isna(row['startShortTrade']) and trade_size == 0:
      shortTradeSize = shares * pot_percentage_to_bet
      shares -= shortTradeSize
      cash_value = shortTradeSize * price * (1 - transcaction_costs)
      cash += cash_value
      is_long = False
      trade_value = cash_value
      trade_price = price

      activity_log.append(f'{index} - Opened short trade for {shortTradeSize} shares at {price}. Cashed £{cash_value / 100}, current cash is £{cash / 100} and current shares are {shares}')
    else:
      continue

  value = cash +  (shares * last_price) * (1 - transcaction_costs)
  initial_value_gbp = initial_value / 100
  final_value_gbp = value / 100
  portfolio_growth_perc = ((value - initial_value) / initial_value) * 100
  asset_growth_perc = ((last_price - first_price) / first_price) * 100
  if trade_count > 0:
    win_percentage = (win_count / trade_count) * 100
  else:
    win_percentage = 0

  activity_log.append('---------------------')
  activity_log.append(f'Initial value: {initial_value_gbp}')
  activity_log.append(f'Final value: {final_value_gbp}')
  activity_log.append(f'Number of trades: {trade_count}')
  activity_log.append(f'Win percentage: {win_percentage} %')
  activity_log.append(f'Portfolio growth: {portfolio_growth_perc} %')
  activity_log.append(f'Asset growth: {asset_growth_perc} %')  

  return StrategyReturns(
    initial_value_gbp,
    final_value_gbp,
    portfolio_growth_perc,
    asset_growth_perc,
    trade_count,
    win_percentage,
    activity_log,
  )

In [671]:
import copy

def run_lorentzian_strategy_test(data_settings: DataSettings, prediction_settings: LorentzianSettings, strategy_settings: StrategySettings) -> LorentzianPredictionResult:
    df = data_settings.get_data(data_settings.symbol, data_settings.interval, data_settings.window, data_settings.skip, data_settings.buffer)

    lc = calculate_predictions(df, prediction_settings)
    # lc.dump(f'{data_folder}/{data_settings.symbol}_{data_settings.interval}_{data_settings.window}_{data_settings.skip}_result.csv')
    # lc.plot(f'{data_folder}/{data_settings.symbol}_{data_settings.interval}_{data_settings.window}_{data_settings.skip}_result.jpg')

    perf_period = lc.data.tail(data_settings.window)
    returns =  calculate_performance(perf_period, strategy_settings)

    activity_filename = f'trades-log_{data_settings.symbol}_{data_settings.interval}_{data_settings.window}_{data_settings.skip:03}_{data_settings.buffer}.txt'
    with open(f'{data_folder}/{activity_filename}', 'w') as file:
      for line in returns.activity_log:
          file.write(line + "\n")
    
    returns.activity_log = activity_filename

    return LorentzianPredictionResult(copy.deepcopy(data_settings), copy.deepcopy(prediction_settings), returns)

In [672]:
period = 365
buffer_periods = 3
buffer = buffer_periods * period
num_of_periods = 10 - buffer_periods

data_settings = DataSettings(
    symbol = 'BTC-GBP', interval = '1d', window = period, skip = 0, buffer=buffer, get_data=get_data_from_yfinance
)

# data_settings = DataSettings(
#     symbol = 'XBTGBP', interval = '1440', window = period, skip = 0, buffer=buffer, get_data=get_data_from_kraken_csv
# )

prediction_settings = LorentzianSettings(
    # neighborsCount = 8, maxBarsBack = 2000, useDynamicExits = True,
    # useEmaFilter = False, emaPeriod = 200,
    # useSmaFilter = False, smaPeriod = 200,
    # useKernelSmoothing = True, lookbackWindow = 8, relativeWeight = 8.0, regressionLevel = 25, crossoverLag = 2,
    # useVolatilityFilter = True,
    # useRegimeFilter = True, regimeThreshold = -.1,
    # useAdxFilter = False, adxThreshold = 25,
    # use_RSI = True, RSI_param1 = 14, RSI_param2 = 2,
    # use_WT = True, WT_param1 = 10, WT_param2 = 11,
    # use_CCI = True, CCI_param1 = 20, CCI_param2 = 2,
    # use_ADX = True, ADX_param1 = 20, ADX_param2 = 2,
    # use_MFI = True, MFI_param1 = 14
    neighborsCount = 8, maxBarsBack = buffer, useDynamicExits = False,
    useEmaFilter = False, emaPeriod = 200,
    useSmaFilter = False, smaPeriod = 200,
    useKernelSmoothing = False, lookbackWindow = 12, relativeWeight = 8.0, regressionLevel = 30, crossoverLag = 0,
    useVolatilityFilter = False,
    useRegimeFilter = False, regimeThreshold = -1,
    useAdxFilter = True, adxThreshold = 40,
    use_RSI = True, RSI_param1 = 28, RSI_param2 = 7,
    use_WT = True, WT_param1 = 5, WT_param2 = 11,
    use_CCI = False, CCI_param1 = 2, CCI_param2 = 2,
    use_ADX = False, ADX_param1 = 2, ADX_param2 = 2,
    use_MFI = False, MFI_param1 = 14
)

strategy_settings = StrategySettings(
  transcaction_costs = .004, pot_percentage_to_bet = 1.0, initial_cash_percentage = 1, initial_value = 1000.0, stop_loss_percentage=.1
)

optimise_parameter = False
property_to_optimise = 'RSI_param2'
opt_start_value = 1
opt_end_value = 50
opt_step = 1
opt_test = opt_start_value

returns = []

while not optimise_parameter or opt_test < opt_end_value:
  
  for j in range(num_of_periods):
    returns.append(run_lorentzian_strategy_test(data_settings, prediction_settings, strategy_settings))
    data_settings.skip += 1

  if optimise_parameter:
    opt_test += opt_step
    setattr(prediction_settings, property_to_optimise, opt_test)
  else:
    break
    
dicts = [{**vars(r.data_settings), **vars(r.returns), **vars(r.settings)} for r in returns]
perf = pd.DataFrame.from_dict(dicts)

wins_average = perf['win_percentage'].mean()

returns_average = perf['portfolio_growth'].mean()
returns_min = perf['portfolio_growth'].min()
returns_max = perf['portfolio_growth'].max()

asset_average =  perf['asset_growth'].mean()
asset_min =  perf['asset_growth'].min()
asset_max =  perf['asset_growth'].max()

print(f'Wins: {wins_average} %')
print('\n----------------\n')
print(f'Average return: {returns_average} %')
print(f'Average asset return: {asset_average} %')
print('\n----------------\n')
print(f'Worst period: {returns_min} %')
print(f'Asset worst period: {asset_min} %')
print('\n----------------\n')
print(f'Best period: {returns_max} %')
print(f'Asset best period: {asset_max} %')

Wins: 45.0 %

----------------

Average return: -3.1560952337788604 %
Average asset return: 80.08551005391587 %

----------------

Worst period: -14.116247423779729 %
Asset worst period: -9.05469495961445 %

----------------

Best period: 8.17861166611615 %
Asset best period: 208.78170064644456 %


In [673]:
import pandas as pd
import matplotlib.pyplot as plt

benchmark_field = 'portfolio_growth'

if optimise_parameter:
    grouped = perf.groupby(property_to_optimise)[benchmark_field].mean().reset_index()

    # Plotting the scatter plot
    plt.scatter(grouped[property_to_optimise], grouped[benchmark_field])

    # Adding labels and title
    plt.xlabel(property_to_optimise)
    plt.ylabel(benchmark_field)
    plt.title(f'Scatter Plot of {benchmark_field} by {property_to_optimise}')

    # Display the plot
    plt.show()

In [674]:
page = 0
items_per_page = 12

perf[['portfolio_growth', 'asset_growth', 'trade_count', 'win_percentage', 'activity_log']][page * items_per_page: (page * items_per_page) + items_per_page]

,portfolio_growth,asset_growth,trade_count,win_percentage,activity_log
0,-1.096611,161.319377,5,60.0,trades-log_BTC-GBP_1d_365_000_1095.txt
1,-0.662564,-9.054695,2,50.0,trades-log_BTC-GBP_1d_365_001_1095.txt
2,-6.274784,4.785457,1,0.0,trades-log_BTC-GBP_1d_365_002_1095.txt
3,-8.943253,208.781701,10,30.0,trades-log_BTC-GBP_1d_365_003_1095.txt
4,-14.116247,26.802107,4,50.0,trades-log_BTC-GBP_1d_365_004_1095.txt
5,8.178612,11.628058,4,75.0,trades-log_BTC-GBP_1d_365_005_1095.txt
6,0.822180,156.336566,2,50.0,trades-log_BTC-GBP_1d_365_006_1095.txt
